In [ ]:
from audio_midi_pipeline import process_files
import pandas as pd
import pretty_midi
import librosa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
file_path_test = "songs/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_08_Track08_wav."
df_test = process_files(file_path_test)
labels = df_test.iloc[:, 513:]
inputs = df_test.iloc[:, :513]

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset

class SpectrogramDataset(Dataset):
    def __init__(self, inputs_df, labels_df=None):
        self.inputs = torch.FloatTensor(inputs_df.values)
        self.has_labels = labels_df is not None
        self.labels = torch.FloatTensor(labels_df.values) if self.has_labels else None

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_tensor = self.inputs[idx].view(1, 1, 513)  # [channels, time, freq]
        if self.has_labels:
            return input_tensor, self.labels[idx]
        return input_tensor

class PitchDetectionModel(nn.Module):
    def __init__(self, num_pitches=88):
        super(PitchDetectionModel, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(1, 3), padding=(0, 1)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((1, 2)),

            nn.Conv2d(32, 64, kernel_size=(1, 3), padding=(0, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((1, 2)),

            nn.Conv2d(64, 128, kernel_size=(1, 3), padding=(0, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 128, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_pitches),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Load the saved model
model = torch.load('full_model.pth')
model.eval()

# Load your new input data
new_inputs = inputs

# Create a dataset and dataloader for the new input data
dataset = SpectrogramDataset(new_inputs)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Make predictions on the new data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

with torch.no_grad():
    for data in dataloader:
        data = data.to(device)
        outputs = model(data)
        # Do something with the model outputs, e.g., save predictions
        print(outputs)